# Train and deploy on Google Cloud Platform (GCP)

This notebook introduces you to using Kubeflow Fairing to train and deploy a model to Kubeflow on Google Kubernetes Engine (GKE), and Google Cloud ML Engine. This notebook demonstrate how to:
 
* Train an XGBoost model in a local notebook,
* Use Kubeflow Fairing to train an XGBoost model remotely on Kubeflow,
* Use Kubeflow Fairing to train an XGBoost model remotely on Cloud ML Engine,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow, and
* Call the deployed endpoint for predictions.

To learn more about how to run this notebook locally, see the guide to [training and deploying on GCP from a local notebook][gcp-local-notebook].

[gcp-local-notebook]: https://kubeflow.org/docs/fairing/gcp-local-notebook/

## Set up your notebook for training an XGBoost model

Import the libraries required to train this model.

In [8]:
!pip install joblib
!pip install pandas
!pip install scikit-learn

You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 5.4MB 4.7MB/s eta 0:00:01
You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import argparse
import logging
import joblib
import sys
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

In [2]:
logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

Define a function to split the input file into training and testing datasets.

In [3]:
def read_input(file_name, test_size=0.25):
    """Read input data and split it into train and test."""
    data = pd.read_csv(file_name)
    data.dropna(axis=0, subset=['SalePrice'], inplace=True)

    y = data.SalePrice
    X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

    train_X, test_X, train_y, test_y = train_test_split(X.values,
                                                      y.values,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)

Define functions to train, evaluate, and save the trained model.

Define a class for your model, with methods for training and prediction.

In [4]:
class HousingServe(object):
    
    def __init__(self):
        self.train_input = "ames_dataset/train.csv"
        self.n_estimators = 50
        self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None

    def train(self):
        (train_X, train_y), (test_X, test_y) = read_input(self.train_input)
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

    def predict(self, X, feature_names):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return [[prediction.item(0), prediction.item(0)]]

## Set up Kubeflow Fairing for training and predictions on GCP

Import the `fairing` library and configure the GCP environment that your training or prediction job will run in.

In [6]:
import os
import fairing

# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(GCP_PROJECT)
PY_VERSION = ".".join([str(x) for x in sys.version_info[0:3]])
BASE_IMAGE = 'python:{}'.format(PY_VERSION)

## Deploy the trained model to Kubeflow for predictions

Import the `PredictionEndpoint` and `KubeflowGKEBackend` classes. Kubeflow Fairing packages the `HousingServe` class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [8]:
from fairing import PredictionEndpoint
from fairing.backends import KubeflowGKEBackend
endpoint = PredictionEndpoint(HousingServe, BASE_IMAGE, input_files=['trained_ames_model.dat', "requirements.txt"],
                              docker_registry=DOCKER_REGISTRY, backend=KubeflowGKEBackend())
endpoint.create()

Using preprocessor: <class 'fairing.preprocessors.function.FunctionPreProcessor'>
Using docker registry: gcr.io/code-search-demo/fairing-job
Using builder: <class 'fairing.builders.cluster.cluster.ClusterBuilder'>
/opt/conda/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
/opt/conda/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-nmsd7 to start...
Waiting for fairing-builder-nmsd7 to start...
Pod started running True


INFO[0000] Downloading base image python:3.6.7
ERROR: logging before flag.Parse: E0410 12:01:31.748413       1 metadata.go:142] while reading 'google-dockercfg' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg
ERROR: logging before flag.Parse: E0410 12:01:31.751884       1 metadata.go:159] while reading 'google-dockercfg-url' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg-url
2019/04/10 12:01:31 No matching credentials were found, falling back on anonymous
INFO[0000] Executing 0 build triggers
INFO[0000] Unpacking rootfs as cmd RUN if [ -e requirements.txt ];then pip install --no-cache -r requirements.txt; fi requires it.
INFO[0020] Taking snapshot of full filesystem...
INFO[0028] Skipping paths under /dev, as it is a whitelisted directory
INFO[0028] Skipping paths under /etc/secrets, as it is a whitelist

  Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'
  Running setup.py install for tornado: started
    Running setup.py install for tornado: finished with status 'done'
  Running setup.py install for threadloop: started
    Running setup.py install for threadloop: finished with status 'done'
  Running setup.py install for thrift: started
    Running setup.py install for thrift: finished with status 'done'
  Running setup.py install for opentracing: started
    Running setup.py install for opentracing: finished with status 'done'
  Running setup.py install for jaeger-client: started
    Running setup.py install for jaeger-client: finished with status 'done'
  Running setup.py install for absl-py: started
    Running setup.py install for absl-py: finished with status 'done'
  Running setup.py install for termcolor: started
    Running setup.py install for termcolor: finished with status 'done'
  Running setup.py install f

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
INFO[0109] Taking snapshot of full filesystem...
INFO[0113] Skipping paths under /dev, as it is a whitelisted directory
INFO[0113] Skipping paths under /etc/secrets, as it is a whitelisted directory
INFO[0113] Skipping paths under /kaniko, as it is a whitelisted directory
INFO[0113] Skipping paths under /proc, as it is a whitelisted directory
INFO[0113] Skipping paths under /sys, as it is a whitelisted directory
INFO[0113] Skipping paths under /var/run, as it is a whitelisted directory
INFO[0147] Using files from context: [/kaniko/buildcontext/app]
INFO[0147] COPY /app/ /app/
INFO[0147] Taking snapshot of files...
2019/04/10 12:03:59 existing blob: sha256:087a57faf9491b1b82a83e26bc8cc90c90c30e4a4d858b57ddd5b4c2c90095f6
2019/04/10 12:03:59 existing blob: sha256:687ed2fb2a0d7da5503478759fd00c23970b65d02b317119b3fb9025038a2594
2019/04/10 12:03:59 

Endpoint fairing-deployer-6r5zz launched.


Waiting for prediction endpoint to come up...


Prediction endpoint: http://34.73.17.74:5000/predict


## Call the prediction endpoint

Create a test dataset, then call the endpoint on Kubeflow for predictions.

In [10]:
(train_X, train_y), (test_X, test_y) = read_input("/home/jovyan/git_kubeflow-fairing/examples/prediction/ames_dataset/train.csv")
endpoint.predict_nparray(test_X)


{"data":{"names":["t:0","t:1"],"tensor":{"shape":[1,2],"values":[165164.875,165164.875]}},"meta":{}}



## Clean up the prediction endpoint

Delete the prediction endpoint created by this notebook.

In [18]:
endpoint.delete()

Deleted service: kubeflow/fairing-service-722gw
Deleted deployment: kubeflow/fairing-deployer-tb6p4
